## Importing files and libraries

In [ ]:
!gdown 12Z1bFBPRsEYFswdEMGzyTnthL4xEdZl-
!unrar x /content/images.rar

In [2]:
!mkdir images
!mv ./test ./images/test
!mv ./validation ./images/validation
!mv ./train ./images/train

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import copy
import os
from tqdm import tqdm
import torchvision.transforms as transforms
from PIL import Image
import shutil
from google.colab import drive

## Preproceesing

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'validation': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

In [5]:
data_dir = "./images"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'validation', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'validation', 'test']}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_names = image_datasets['train'].classes

## Training the model

In [21]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    model.to(device)
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Training phase
        model.train()
        train_loss = 0.0
        train_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], total=len(dataloaders['train'])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_corrects += torch.sum(preds == labels.data)

        train_loss = train_loss / len(image_datasets['train'])
        train_acc = train_corrects.double() / len(image_datasets['train'])
        print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.2%}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        for inputs, labels in tqdm(dataloaders['validation'], total=len(dataloaders['validation'])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(image_datasets['validation'])
        val_acc = val_corrects.double() / len(image_datasets['validation'])
        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.2%}')

        # Check if current model has the best validation accuracy
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print(f'Best val Acc: {best_acc:.2%}')
    model.load_state_dict(best_model_wts)
    return model


In [22]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [23]:
model = train_model(model, criterion, optimizer, scheduler, num_epochs=5)


Epoch 1/5
----------


100%|██████████| 13771/13771 [20:13<00:00, 11.35it/s]


Train Loss: 0.2035 Acc: 93.53%


100%|██████████| 1722/1722 [00:48<00:00, 35.25it/s]


Val Loss: 0.0130 Acc: 99.49%

Epoch 2/5
----------


...

Val Loss: 0.0095 Acc: 99.81%

Epoch 5/5
----------


100%|██████████| 13771/13771 [20:43<00:00, 11.07it/s]


Train Loss: 0.0066 Acc: 99.83%


100%|██████████| 1722/1722 [00:50<00:00, 33.83it/s]


Val Loss: 0.0066 Acc: 99.78%

Best val Acc: 99.91%


## Testing the model

In [24]:
def test_model(model):
    model.to(device)
    model.eval()
    running_corrects = 0

    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / len(image_datasets['test'])
    print(f'Test Accuracy: {test_acc:.4f}')

test_model(model)


Test Accuracy: 0.9990


## Test with external Image

In [34]:
def test_external_image(model, image_path):
    # Load and preprocess the image
    image_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    image = Image.open(image_path).convert("RGB")
    image = image_transforms(image)
    image = image.unsqueeze(0)

    # Move the image tensor to the same device as the model
    image = image.to(device)

    # Pass the image through the model
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    # Get the predicted class index and class name
    predicted_class_index = predicted.item()
    predicted_class_name = class_names[predicted_class_index]

    # Print the predicted class name
    print(f"Predicted class: {predicted_class_name}")

# Example usage
image_path = "/ccc.jpg"  # Replace with the path to your image
test_external_image(model, image_path)


Predicted class: Yasamin


## Saving Model

In [25]:
torch.save(model.state_dict(), 'new_model.pth')

## Load model

In [ ]:
# Create an instance of the model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
num_classes = len(class_names)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load the saved model state dictionary
model.load_state_dict(torch.load('/content/Trained_ResNet101.pth'))

# Set the model to evaluation mode
model.eval()